### Part 4: Retention analysis. Churn rate

Before we calculate, vizualize and compare retention rate across different user groups, let's first see weekly:
  - how many users are new, how many users been in the feed both current and last week, how many users been active last, but not current week? 
  
So the groups are:

  - new users: start their activity of the current week
  - retained users: active both of the current and previous weeks
  - gone users: been active on previous week, but not on the current week

In [1]:
# start with the imports we would need
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.axes as axes
from scipy.stats import chi2_contingency # Chi-Squared testing
import pandas as pd
import pandahouse as ph # Connect with clickhouse DB
import seaborn as sns


In [35]:
# set quality of the plots to be built
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

# Set the scaling factors for the plots
SCALE = 1 / 6
sns.set(font_scale=1 * SCALE)


In [36]:
# set the connection with the db
CONNECTION = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'simulator_20230720',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }


In [37]:
# get a table where for each week we count:
# 1. new users;
# 2. users active this and previous week;
# 3. users active this, but not following week.
query = '''
-- table where for every unique pair user-active week, 
-- shows if this user joined that week, was active last week, and was active next week
WITH weeks_labeled AS (
SELECT 
user_id, 
week, 
all_weeks,
start_week,
if(has(all_weeks, week - 7), 1, 0) AS is_previous_week,
if(has(all_weeks, week + 7), 1, 0) AS is_next_week

FROM

-- table showing all user activity weeks aggregated into an array
(SELECT DISTINCT user_id, 
toStartOfWeek(time) AS week, 
groupUniqArray(toStartOfWeek(time)) OVER (PARTITION BY user_id) AS all_weeks,
min(toStartOfWeek(time)) OVER (PARTITION BY user_id) AS start_week
FROM simulator_20230720.feed_actions) t1
)

SELECT toString(week) as week,
new_users,
retained_users,
gone_users

FROM 
-- table having counts for new and retained users by week
(SELECT 
week, 
countIf(week = start_week) AS new_users,
countIf(is_previous_week = 1) AS retained_users
FROM weeks_labeled
GROUP BY week
) with_retained

JOIN 

-- table having counts for users who didn`t come from last week.
-- basically, weeks are shifted one week up to align 
-- with is_next_week column to indicate how many users did not come ("gone")
(SELECT 
week + 7 AS to_next_week,
countIf(is_next_week = 0) * (-1) AS gone_users
FROM weeks_labeled
GROUP BY to_next_week
) with_gone

ON (with_retained.week = with_gone.to_next_week)
'''

# import a table to a dataframe
weeks_by_user_groups = ph.read_clickhouse(
    query, 
    connection=CONNECTION
)
